In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 128, 3, padding=1)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(128 * 4 * 4, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, 100)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = self.pool(torch.nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

num_epochs = 175
num_runs = 5

all_train_losses = []
all_train_accuracies = []
all_test_accuracies = []

checkpoint_dir = 'checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

def save_checkpoint(run, epoch, model, optimizer, train_losses, train_accuracies, test_accuracies):
    checkpoint = {
        'run': run,
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
    }
    filename = os.path.join(checkpoint_dir, f'checkpoint_run{run}_epoch{epoch}.pth')
    torch.save(checkpoint, filename)
    print(f'Saved checkpoint: {filename}')

for run in range(num_runs):
    print(f"Starting run {run + 1}/{num_runs}")
    net = Net()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    train_losses = []
    train_accuracies = []
    test_accuracies = []
    for epoch in range(num_epochs):
        net.train()
        correct = 0
        total = 0
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = torch.nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = 100 * correct / total
        avg_loss = running_loss / len(trainloader)
        train_accuracies.append(train_accuracy)
        train_losses.append(avg_loss)
        print(f"Run {run + 1}, Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

        scheduler.step()
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        test_accuracies.append(test_accuracy)
        print(f"Run {run + 1}, Epoch {epoch + 1}, Test Accuracy: {test_accuracy:.2f}%")
        if (epoch + 1) % 10 == 0:
            save_checkpoint(run, epoch + 1, net, optimizer, train_losses, train_accuracies, test_accuracies)
    all_train_losses.append(train_losses)
    all_train_accuracies.append(train_accuracies)
    all_test_accuracies.append(test_accuracies)
    print(f"Final Training Losses for Run {run + 1}: {train_losses}")
    print(f"Final Training Accuracies for Run {run + 1}: {train_accuracies}")
    print(f"Final Test Accuracies for Run {run + 1}: {test_accuracies}")
    save_checkpoint(run, num_epochs, net, optimizer, train_losses, train_accuracies, test_accuracies)
np.savetxt('all_train_losses.txt', np.array(all_train_losses), fmt='%f')
np.savetxt('all_train_accuracies.txt', np.array(all_train_accuracies), fmt='%f')
np.savetxt('all_test_accuracies.txt', np.array(all_test_accuracies), fmt='%f')
print("All Training Losses over Epochs for each run:", all_train_losses)
print("All Training Accuracies over Epochs for each run:", all_train_accuracies)
print("Final Test Accuracies for each run:", all_test_accuracies)
